In [1]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

In [2]:
# Pkg.add("NBInclude")
using NBInclude
# Pkg.add("Chess")
using Chess
using Test
using BenchmarkTools

In [3]:
@nbinclude("EvaluatePosition.ipynb")

terminal_evaluation (generic function with 1 method)

In [4]:
@nbinclude("RandomChess.ipynb")

playRandomMove (generic function with 1 method)

# Evaluate Position Testing

## Test CalculatePosition for a single piece



In [5]:
startBoard=startboard()
queen = Piece(WHITE, QUEEN)
wPawn = Piece(WHITE, PAWN)
bPawn = Piece(BLACK, PAWN)
sD6 = Square(FILE_D, RANK_6)
sD2 = Square(FILE_D, RANK_2)
sD7 = Square(FILE_D, RANK_7)
b1 = fromfen("rnbqkbnr/ppp1pppp/8/3p4/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 1")
sD5 = Square(FILE_D, RANK_5)
@testset "Get Value of Piece" begin
    display(startBoard)
    @test calc_score(startBoard, queen, sD6) == 905
    @test calc_score(startBoard, wPawn, sD2) == 80 #100 pawn Value and -20 square 
    @test calc_score(startBoard, bPawn, sD7) == 80 #100 pawn Value and -20 square 
    display(b1)
    @test calc_score(b1, bPawn, sD5) == 120
end

Board (rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  P  P  P  P 
 R  N  B  Q  K  B  N  R

Board (rnbqkbnr/ppp1pppp/8/3p4/4P3/8/PPPP1PPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  -  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  p  -  -  -  - 
 -  -  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Test Summary:      | Pass  Total  Time
Get Value of Piece |    4      4  4.2s


Test.DefaultTestSet("Get Value of Piece", Any[], 4, false, false, true, 1.678620826727e9, 1.678620830963e9)

## Evaluation of position 

In [6]:
b1 = fromfen("rnbqkbnr/ppp1pppp/8/3P4/8/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1")
b2 = fromfen("rn1qkb1r/pppb1ppp/4pn2/3p4/3P1B2/2N5/PPPQPPPP/R3KBNR w KQkq - 2 5")
b3 = fromfen("1Q6/1R6/1B4k1/1N4p1/1P4n1/1K4b1/6r1/6q1 w - - 0 1")

@testset "Get Value of Piece" begin
    display(b1)
    @test evaluate_position(b1) == 125
    display(b2)
    @test evaluate_position(b2) == -5
    display(b3)
    @test evaluate_position(b3) == 0
end

Board (rnbqkbnr/ppp1pppp/8/3P4/8/8/PPPP1PPP/RNBQKBNR b KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  -  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  P  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Board (rn1qkb1r/pppb1ppp/4pn2/3p4/3P1B2/2N5/PPPQPPPP/R3KBNR w KQkq -):
 r  n  -  q  k  b  -  r 
 p  p  p  b  -  p  p  p 
 -  -  -  -  p  n  -  - 
 -  -  -  p  -  -  -  - 
 -  -  -  P  -  B  -  - 
 -  -  N  -  -  -  -  - 
 P  P  P  Q  P  P  P  P 
 R  -  -  -  K  B  N  R

Board (1Q6/1R6/1B4k1/1N4p1/1P4n1/1K4b1/6r1/6q1 w - -):
 -  Q  -  -  -  -  -  - 
 -  R  -  -  -  -  -  - 
 -  B  -  -  -  -  k  - 
 -  N  -  -  -  -  p  - 
 -  P  -  -  -  -  n  - 
 -  K  -  -  -  -  b  - 
 -  -  -  -  -  -  r  - 
 -  -  -  -  -  -  q  -

Test Summary:      | Pass  Total  Time
Get Value of Piece |    3      3  0.8s


Test.DefaultTestSet("Get Value of Piece", Any[], 3, false, false, true, 1.67862083186e9, 1.678620832658e9)

## Test Capture Piece 

In [7]:
sD5 = Square(FILE_D,RANK_5)
sA8 = Square(FILE_A,RANK_8)
b1 = fromfen("rnbqkbnr/ppp1pppp/8/3p4/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 1")
b2 = fromfen("rnbqkb1r/pP2pp1p/5np1/8/8/8/PPPP1PPP/RNBQKBNR w KQkq - 1 5")
@testset "Capture Piece" begin
    display(b1)
    @test valueCapturePiece(b1, sD5) == 120
    display(b2)
    @test valueCapturePiece(b2, sA8) == 500
end

Board (rnbqkbnr/ppp1pppp/8/3p4/4P3/8/PPPP1PPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  -  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  p  -  -  -  - 
 -  -  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Board (rnbqkb1r/pP2pp1p/5np1/8/8/8/PPPP1PPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  -  r 
 p  P  -  -  p  p  -  p 
 -  -  -  -  -  n  p  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Test Summary: | Pass  Total  Time
Capture Piece |    2      2  0.0s


Test.DefaultTestSet("Capture Piece", Any[], 2, false, false, true, 1.67862083267e9, 1.678620832688e9)

## Test En Passant

In [8]:
b1=fromfen("rnbqkbnr/ppp1pppp/8/3pP3/8/8/PPPP1PPP/RNBQKBNR b KQkq - 0 2")
domove!(b1, "f5")
sF6 = Square(FILE_F,RANK_6)
@testset "checkEnPassant" begin
    display(b1)
    @test checkEnPassant(b1, sF6) == 100
end

Board (rnbqkbnr/ppp1p1pp/8/3pPp2/8/8/PPPP1PPP/RNBQKBNR w KQkq f6):
 r  n  b  q  k  b  n  r 
 p  p  p  -  p  -  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  p  P  p  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Test Summary:  | Pass  Total  Time
checkEnPassant |    1      1  0.1s


Test.DefaultTestSet("checkEnPassant", Any[], 1, false, false, true, 1.678620834497e9, 1.6786208346e9)

In [9]:
b1=fromfen("rn1qkb1r/pppb1ppp/4pn2/3p4/3P1B2/2N5/PPPQPPPP/R3KBNR w KQkq - 2 5")
sC1 = Square(FILE_C,RANK_1)
piece = Piece(WHITE, KING)
@testset "calc_castle" begin
    display(board)
    @test calc_castle(b1, piece ,sC1) == 5
end

board (generic function with 2 methods)

Test Summary: | Pass  Total  Time
calc_castle   |    1      1  0.2s


Test.DefaultTestSet("calc_castle", Any[], 1, false, false, true, 1.678620834609e9, 1.678620834806e9)

## Testing IsEndGame


In [10]:
b1 = fromfen("2bqk3/8/8/8/8/8/5B2/3QK3 w - - 0 1")
b2 = fromfen("rnb1kbnr/ppp2ppp/8/8/8/8/PPP2PPP/RNB1KBNR w KQkq - 0 1")
b3 = fromfen("2b1kb2/pp4pp/8/8/8/8/PP4P1/R2QK3 w Q - 0 1")
b4 = fromfen("r1b1kbnr/p2p1ppp/2pP4/1p2p1B1/4P3/1B3P2/PPP2P1P/RN3RK1 w kq - 0 1")
@testset "Testing Endgame" begin
    display(b1)
    @test is_endgame(b1) == true
    display(b2)
    @test is_endgame(b2) == true
    display(b3)
    @test is_endgame(b3) == false
    display(b4)
    @test is_endgame(b4) == true
end

Board (2bqk3/8/8/8/8/8/5B2/3QK3 w - -):
 -  -  b  q  k  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  B  -  - 
 -  -  -  Q  K  -  -  -

Board (rnb1kbnr/ppp2ppp/8/8/8/8/PPP2PPP/RNB1KBNR w KQkq -):
 r  n  b  -  k  b  n  r 
 p  p  p  -  -  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  -  -  P  P  P 
 R  N  B  -  K  B  N  R

Board (2b1kb2/pp4pp/8/8/8/8/PP4P1/R2QK3 w Q -):
 -  -  b  -  k  b  -  - 
 p  p  -  -  -  -  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  -  -  -  -  P  - 
 R  -  -  Q  K  -  -  -

Board (r1b1kbnr/p2p1ppp/2pP4/1p2p1B1/4P3/1B3P2/PPP2P1P/RN3RK1 w kq -):
 r  -  b  -  k  b  n  r 
 p  -  -  p  -  p  p  p 
 -  -  p  P  -  -  -  - 
 -  p  -  -  p  -  B  - 
 -  -  -  -  P  -  -  - 
 -  B  -  -  -  P  -  - 
 P  P  P  -  -  P  -  P 
 R  N  -  -  -  R  K  -

Test Summary:   | Pass  Total  Time
Testing Endgame |    4      4  0.0s


Test.DefaultTestSet("Testing Endgame", Any[], 4, false, false, true, 1.678620834832e9, 1.678620834869e9)

## Evaluate Move for terminal positions

In [11]:
b1 = fromfen("7k/8/8/3K4/8/6R1/8/5R2 w - - 0 1")
m1 = movefromstring("f1h1")
b2 = fromfen("k7/7K/8/8/6r1/8/8/5r2 b - - 0 1")
m2 = movefromstring("f1h1")
@testset "Evaluate Move terminal position" begin
    display(b1)
    @test evaluate_move(b1, m1, evaluate_position(b1)) == evaluate_position(domove(b1, m1))
    display(b2)
    @test evaluate_move(b2, m2, evaluate_position(b2)) == evaluate_position(domove(b2, m2))
end

Board (7k/8/8/3K4/8/6R1/8/5R2 w - -):
 -  -  -  -  -  -  -  k 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  K  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  R  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  R  -  -

Board (k7/7K/8/8/6r1/8/8/5r2 b - -):
 k  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  K 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  r  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  r  -  -

Test Summary:                   | Pass  Total  Time
Evaluate Move terminal position |    2      2  0.4s


Test.DefaultTestSet("Evaluate Move terminal position", Any[], 2, false, false, true, 1.678620834891e9, 1.678620835304e9)

## Evaluate Move for endgame transition

In [12]:
b1 = fromfen("r1b1kbnr/p2p1ppp/2pP4/1p2p1B1/4P3/1B3q2/PPP2PPP/RN3RK1 w kq - 0 1")
m1 = movefromstring("g2f3")
@testset "evaluate_move" begin
    display(b1)
    @test evaluate_move(b1, m1, evaluate_position(b1)) == evaluate_position(domove(b1, m1)) # regular position
end

Board (r1b1kbnr/p2p1ppp/2pP4/1p2p1B1/4P3/1B3q2/PPP2PPP/RN3RK1 w kq -):
 r  -  b  -  k  b  n  r 
 p  -  -  p  -  p  p  p 
 -  -  p  P  -  -  -  - 
 -  p  -  -  p  -  B  - 
 -  -  -  -  P  -  -  - 
 -  B  -  -  -  q  -  - 
 P  P  P  -  -  P  P  P 
 R  N  -  -  -  R  K  -

Test Summary: | Pass  Total  Time
evaluate_move |    1      1  0.0s


Test.DefaultTestSet("evaluate_move", Any[], 1, false, false, true, 1.678620835324e9, 1.678620835326e9)

## Evaluate Move

In [13]:
# white to move
b1 = startboard()
m1 = Move(Square(FILE_E,RANK_2), Square(FILE_E,RANK_4))

# Capture white and promotion
b2 = fromfen("rnbqkb1r/pP2pp1p/5np1/8/8/8/PPPP1PPP/RNBQKBNR w KQkq - 1 5")
m2 = Move(Square(FILE_B,RANK_7), Square(FILE_A,RANK_8), QUEEN)

# black to move
b3 = fromfen("7k/P7/8/8/8/8/8/K7 b - - 0 1")
m3 = movefromstring("h8h7")
b4 = fromfen("rnbqkbnr/ppp2ppp/8/3Pp3/8/5N2/PPPP1PPP/RNBQKB1R b KQkq - 0 1")
m4 = movefromstring("d8h4")

# Captures white
b5 = fromfen("rnb1kbnr/ppp2ppp/8/3Pp3/7q/5N2/PPPP1PPP/RNBQKB1R w KQkq - 0 1")
m5 = movefromstring("f3h4")

@testset "evaluate_move" begin
    display(domove(b1, m1))
    @test evaluate_move(b1, m1, evaluate_position(b1)) == evaluate_position(domove(b1, m1)) # regular position
    display(domove(b2, m2))
    @test evaluate_move(b2, m2, evaluate_position(b2)) == evaluate_position(domove(b2, m2))
    display(domove(b3, m3))
    @test evaluate_move(b3, m3, evaluate_position(b3)) == evaluate_position(domove(b3, m3)) # black king movement
    display(domove(b4, m4))
    @test evaluate_move(b4, m4, evaluate_position(b4)) == evaluate_position(domove(b4, m4)) # Queen blunder black
    display(domove(b5, m5))
    @test evaluate_move(b5, m5, evaluate_position(b5)) == evaluate_position(domove(b5, m5))
end
@btime evaluate_move(b5, m5, evaluate_position(b5)) == evaluate_position(domove(b5, m5))

Board (rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Board (Qnbqkb1r/p3pp1p/5np1/8/8/8/PPPP1PPP/RNBQKBNR b KQk -):
 Q  n  b  q  k  b  -  r 
 p  -  -  -  p  p  -  p 
 -  -  -  -  -  n  p  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

Board (8/P6k/8/8/8/8/8/K7 w - -):
 -  -  -  -  -  -  -  - 
 P  -  -  -  -  -  -  k 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 K  -  -  -  -  -  -  -

Board (rnb1kbnr/ppp2ppp/8/3Pp3/7q/5N2/PPPP1PPP/RNBQKB1R w KQkq -):
 r  n  b  -  k  b  n  r 
 p  p  p  -  -  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  P  p  -  -  - 
 -  -  -  -  -  -  -  q 
 -  -  -  -  -  N  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  -  R

Board (rnb1kbnr/ppp2ppp/8/3Pp3/7N/8/PPPP1PPP/RNBQKB1R b KQkq -):
 r  n  b  -  k  b  n  r 
 p  p  p  -  -  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  P  p  -  -  - 
 -  -  -  -  -  -  -  N 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  -  R

Test Summary: | Pass  Total  Time
evaluate_move |    5      5  0.0s
  75.900 μs (757 allocations: 66.02 KiB)


true

### Evaluate move - Promotion

In [14]:
bw = fromfen("8/pk2P3/8/8/8/8/PK6/8 w - - 0 1")
mwN = movefromstring("e7e8N")    # mwN = move white kNight
mwB = movefromstring("e7e8B")
mwR = movefromstring("e7e8R")
mwQ = movefromstring("e7e8Q")

bb = fromfen("8/pk6/8/8/8/8/PK2p3/8 b - - 0 1")
mbN = movefromstring("e2e1N")
mbB = movefromstring("e2e1B")
mbR = movefromstring("e2e1R")
mbQ = movefromstring("e2e1Q")

@testset "evaluate_move_position" begin
    display(bw)
    @test evaluate_move(bw, mwN, evaluate_position(bw)) ==  evaluate_position(domove(bw, mwN)) # promotion white Knight
    @test evaluate_move(bw, mwB, evaluate_position(bw)) ==  evaluate_position(domove(bw, mwB)) # promotion white Bishop
    @test evaluate_move(bw, mwR, evaluate_position(bw)) ==  evaluate_position(domove(bw, mwR)) # promotion white Rook
    @test evaluate_move(bw, mwQ, evaluate_position(bw)) ==  evaluate_position(domove(bw, mwQ)) # promotion white Queen
    display(bb)
    @test evaluate_move(bb, mbN, evaluate_position(bb)) ==  evaluate_position(domove(bb, mbN)) # promotion black Knight
    @test evaluate_move(bb, mbB, evaluate_position(bb)) ==  evaluate_position(domove(bb, mbB)) # promotion black Bishop
    @test evaluate_move(bb, mbR, evaluate_position(bb)) ==  evaluate_position(domove(bb, mbR)) # promotion black Rook
    @test evaluate_move(bb, mbQ, evaluate_position(bb)) ==  evaluate_position(domove(bb, mbQ)) # promotion black Queen
end

Board (8/pk2P3/8/8/8/8/PK6/8 w - -):
 -  -  -  -  -  -  -  - 
 p  k  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  K  -  -  -  -  -  - 
 -  -  -  -  -  -  -  -

Board (8/pk6/8/8/8/8/PK2p3/8 b - -):
 -  -  -  -  -  -  -  - 
 p  k  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  K  -  -  p  -  -  - 
 -  -  -  -  -  -  -  -

Test Summary:          | Pass  Total  Time
evaluate_move_position |    8      8  0.0s


Test.DefaultTestSet("evaluate_move_position", Any[], 8, false, false, true, 1.678620845355e9, 1.678620845362e9)

## Testing with random move engine

In [15]:
@testset "EvaluatePosition with Random Engine" begin
    Random.seed!(123)
    testBoard = startboard()
    testScore = 0
    while !isterminal(testBoard)
        move = generateRandomMove(testBoard)
        incrementalScore = evaluate_move(testBoard, move, testScore)
        domove!(testBoard, move)
        actualScore = evaluate_position(testBoard)
        @test incrementalScore == actualScore
        if incrementalScore != actualScore
            display(testBoard)
            println(lastmove(testBoard))
        end
        testScore = actualScore
    end
end

EvaluatePosition with Random Engine: Test Failed at In[15]:10
  Expression: incrementalScore == actualScore
   Evaluated: 445 == 385
Stacktrace:
 [1] macro expansion
   @ C:\Users\yiwen\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Test\src\Test.jl:464 [inlined]
 [2] macro expansion
   @ .\In[15]:10 [inlined]
 [3] macro expansion
   @ C:\Users\yiwen\AppData\Local\Programs\Julia-1.8.2\share\julia\stdlib\v1.8\Test\src\Test.jl:1357 [inlined]
 [4] top-level scope
   @ .\In[15]:2


Board (1k1n4/1r6/1P6/4pBPp/7P/p3P3/PR3K2/R1q2NN1 w - -):
 -  k  -  n  -  -  -  - 
 -  r  -  -  -  -  -  - 
 -  P  -  -  -  -  -  - 
 -  -  -  -  p  B  P  p 
 -  -  -  -  -  -  -  P 
 p  -  -  -  P  -  -  - 
 P  R  -  -  -  K  -  - 
 R  -  q  -  -  N  N  -

Move(c2c1q)
Test Summary:                       | Pass  Fail  Total  Time
EvaluatePosition with Random Engine |  516     1    517  2.8s


LoadError: [91mSome tests did not pass: 516 passed, 1 failed, 0 errored, 0 broken.[39m

Special case error from endgame to middlegame

In [16]:
b = fromfen("1k1n4/1r6/1P6/4pBPp/7P/p3P3/PRp2K2/R4NN1 b - - 0 1")
println(evaluate_position(b))
move = movefromstring("c2c1q")
println(evaluate_position(domove(b, move)))
evaluate_move(b, move, evaluate_position(b))

1185
385


445